In [14]:
from itertools import combinations
from tqdm import tqdm
import numpy as np
from bisect import bisect

In [21]:
class Game:
    def __init__(self, image: list[list[str]]) -> None:
        self.image = image

    @classmethod
    def from_str(cls, value:str) -> 'Game':
        return cls(image=[list(t) for t in value.strip().split('\n')])
    
    @property
    def empty_rows(self) -> list[int]:
        empty_rows = []
        for i, row in enumerate(self.image):
            if not any(u == '#' for u in row):
                empty_rows.append(i)
        return empty_rows
    
    @property
    def empty_columns(self) -> list[int]:
        empty_cols = []
        for j in range(len(self.image[0])):
            col = [t[j] for t in self.image]
            if not any(u == '#' for u in col):
                empty_cols.append(j)
        return empty_cols
    

    @staticmethod
    def get_scaling_factor(part_2: bool):
        return int(1e6) if part_2 else 1
    
    def get_expanded_image(self, part_2: bool = False):
        # add len(empty_cols) cols and len(empty_rows) rows to image due to expansion
        expanded_image = []
        empty_cols = self.empty_columns
        empty_rows = self.empty_rows
        scaling_factor = self.get_scaling_factor(part_2)
        for i, row in enumerate(self.image):
            new_row = []
            for j, value in enumerate(row):
                if j in empty_cols:
                    new_row += ["."] * scaling_factor
                new_row.append(value)
            expanded_image.append(new_row)
            if i in empty_rows:
                to_add = ['.'] * len(new_row)
                expanded_image += [to_add] * scaling_factor  
        return expanded_image
    
    def get_galaxies(self, image: list[list[str]]) -> list[tuple[int, tuple[int, int]]]:
        coords = [(i, j) for i, row in tqdm(enumerate(image), total=len(image)) for j, t in enumerate(row) if t == '#']
        return [(i + 1, u) for i, u in enumerate(coords)]
    
    def get_galaxies_fast(self, scaling_coeff: int):
        # find positions of galaxies in the expanded universe
        coords = []
        expansion_constant = scaling_coeff - 1 # -1 because we replace emtpy row/column
        empty_columns = self.empty_columns
        empty_rows = self.empty_rows
        for y in range(len(self.image)):
            for x in range(len(self.image[0])):
                if self.image[y][x] == '#':
                    dx = expansion_constant * bisect(empty_columns, x)
                    dy = expansion_constant * bisect(empty_rows, y)
                    coords.append((x + dx, y + dy))
        
        return [(i + 1, u) for i, u in enumerate(coords)] 
            
    @staticmethod
    def find_shorted_paths_in_galaxies(galaxies: list):
        # find shortest path between each galazy pair
        all_steps = []
        for (id1, galaxy1), (id2, galaxy2) in combinations(galaxies, 2):
            nx = abs(galaxy1[0] - galaxy2[0])
            ny = abs(galaxy1[1] - galaxy2[1])
            steps = ny + nx
            all_steps.append([id1, id2, steps])
            
        return all_steps
    
    def solution_part_1(self):
        expanded_image = self.get_expanded_image()
        galaxies = self.get_galaxies(expanded_image)
        paths = self.find_shorted_paths_in_galaxies(galaxies)
        return sum([t[-1] for t in paths])
    
    def solution_part_2(self, scaling_coeff: int = 1000000 ):
        galaxies = self.get_galaxies_fast(scaling_coeff)
        paths = self.find_shorted_paths_in_galaxies(galaxies)
        return sum([t[-1] for t in paths])

In [26]:
puzzle_str = """
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
"""

puzzle_str = """
...................#.............#...........................................#..............................................................
.......................................#.........#..................................................................#.........#........#....
..........#........................................................#................#............#..........................................
...#.................................................#......................................#...............................................
.................#........................................#...............................................#.....#...........................
..............................#...........#.................................................................................................
...............................................#.....................................................................................#......
.............................................................#...........................#..................................................
....#..........#...............................................................#............................................................
................................#................................................................#................#.......................#.
...........................#.............#......................#......................................#....................#...............
#..........#..........#....................................#.............#..................................................................
....................................#..............#..........................................................#........#....................
..........................................................................................#.............................................#...
............................................#.......................#..........................#............................................
.......#...........................................................................#.......................#...................#............
................#...........#.........................#.....................................................................................
................................................#...................................................................#.................#.....
....#....................................................................................#..................................................
............#.......................................................................................#.......................................
...................................#...........................#................................................#...........................
.......#.........#......#....................................................#........................................#.....#...............
......................................................#......................................#..............................................
..#.............................#............#.........................................................#....................................
...........................................................#..........#...........................#........................................#
.............#....................................#.......................................#.........................#...............#.......
.............................#..............................................................................................................
......................#....................#...................#...........#.........#.....................................#................
....................................#..................#.......................................#............#............................#..
..................#.............................#......................................................#....................................
.......#.......................#.......................................#........................................#................#..........
...........................................................................................#................................................
.....................#...........................................................................#........#..................#.......#......
...............................................................#..................#.........................................................
....#.......................................#.............#..........#......................................................................
...........................................................................#............#........................#..........................
.........#...............................................................................................................#..................
..........................#......#..........................................................................................................
..............#..................................#................#.......................................#................................#
..........................................#.................#...................#..................#........................................
.....................................................................................................................................#......
.............................#......................#..................................#..............................#.....................
.....................................#.........................................................................#...........#................
........#..................................................................#................................................................
.#..........................................#........................................................#......................................
....................#..............................................................................................#........................
..........................#........#..........................#......................#...........................................#..........
.......................................................#....................................................................................
................#................................#...........................#...........#.........#...................................#....
......................................................................#.....................................................................
.......................#....................................#.....................#..........#..............................................
........................................#...................................................................................................
..........#..........................................................................................#...........#..........................
...#.........................#....................#.........................................................................................
........................................................................#................................................#..............#...
............................................................................................................................................
.......................#..............................#......#..........................#...................................................
.............................................................................#................................#.......#.....................
............................................................................................................................................
................#..................................#.........................................................................#............#.
...........................#.........#..............................#.....#........#................................................#.......
......................#.............................................................................#......#................................
.........#.....................................#........................................................................#...................
..............#................................................#...............................#............................................
.....#...........................#.....................................................................#....................................
....................................................................................#..............................................#........
............................#..............#.......#......................................#................................................#
...........#.............................................#................#.....#.....................................#.....................
....................#............................................................................#...........................#..............
...................................#.............................................................................#...................#......
....#.........#.....................................................#......................................#................................
.........................#......................#..........#................#...............................................................
...................................................................................#......#.................................................
.................#.............................................#............................................................................
.............................#.....................#.................................................#......................................
.........#.............#..................................................................................#........................#........
.......................................#.................................#.....................#......................#...................#.
...............................................#................................#...........................................................
..#....................................................#............#.............................................#...........#.............
............................................................................................................................................
....................................#..............#............#...........................................................................
...........#...................#........................................#.................#..............#.................................#
....................#...............................................................#...............#.......................................
...........................................#...................................#..............................#.......#.....................
......#..........................................#.........#.........#..........................#................................#..........
..............#.............................................................................................................................
............................................................................................................................................
...#...........................................................#.....................................#.....#..............#.................
............................................................................................................................................
........................................#.......#.........#..................#....................................#.......................#.
.....................................................................................................................................#......
.................#.....#........#..........................................................#.................#..............................
.....................................#.................#.................#...............................................#.....#............
.........................................................................................................#..................................
..................................................................#.........................................................................
..............................................#...........#....................#.........................................................#..
..................................#.........................................................................................................
..........#..........................................................................#.......................#.....#........................
.....#..............#.......#..................................#............................................................................
............................................#.................................................#............................#.......#........
..............#..........................................#.................................................................................#
..........................................................................#..........................#......................................
.................................#..............................................#......#....................................................
#.......#..................................................................................................#.....#......................#...
........................................#..........#........................................................................................
............................................................................................#..............................#................
...........................#..........................................#.....................................................................
....................#..........................#..................................................................................#.........
..........................................................#.............................#.......#.............#.............................
#......#........#...................#..............................#.................................................#......................
.........................#.................#..............................#....................................................#............
..............................#....................................................#................#.......................................
................................................................................................................#........#..................
...........#...........................#............#.......................................................................................
...................#.........................................................................#..............................................
...........................................................................#.........#..........................................#...........
..#.............................#..................................#..................................#.............#.......................
................#........#...............#..................................................................#........................#......
...........................................................................................#................................................
..................................................#......#.....................#............................................................
#.........................................................................................................................#.................
......#............#..........................#.............................................................................................
..............#........................................................................#..........................#.........................
................................#............................................................................................#..............
.......................#.........................#......................#..................#.............................................#..
..#.....................................................#.......................#................#.......#..................................
............................................................................................................................................
..............................................................#.............................................................................
...................#.....................#..........................................................................#..........#............
.........#..........................................................................................#.......................................
..........................#......#...............#.....#....................................................................................
.............#.........................................................#......................#.............................................
............................................................#..................#........................#................................#..
..................#....................................................................................................#........#...........
.....#.....................................................................#.................................#..............................
..................................................#.........................................................................................
.....................................#..............................................#........#......#...............................#.......
............................................................................................................................................
...........................#............................#..........#......................................................#.................
#............#.......#...................................................#.......................#..........................................
"""

In [27]:
game  = Game.from_str(value=puzzle_str)

In [28]:
game.solution_part_1()

100%|██████████| 147/147 [00:00<00:00, 90458.14it/s]


9684228

In [29]:
game.solution_part_2(scaling_coeff=1000000)

483844716556